In [ ]:
from aiida import load_profile,orm, engine, common
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params, import_nc,presettings, code_setup
import aiidalab_widgets_base as awb
from settings import *
from utils import make_query,utils
import yaml
from pathlib import Path

out = widgets.Output()
path_config = Path.cwd()/'config/stash_files.yaml'

In [ ]:
def get_dict_from_filter():
    filter_dict  ={}
    for k,v in sq.site_filter.items():
        filter_={}
        for i in v.children[0].children[0].children:
            f = {}
            for j in range(4):
                f[i.children[j].description] = i.children[j].value
            filter_[i.children[0].value]=f
        filter_dict[k] = filter_
    return filter_dict
   

# Inversion

### Import data

In [ ]:
import_nc = import_nc.Import()
import_nc

### Step 1

In [ ]:
sq = sens_query.SearchSens()
sq

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


### Step 3

In [ ]:
resources = code_setup.CodeSetup(plugin_name='inversion')

account = widgets.Text(
            description="account",
            value = 'em05',
            style = {"description_width": "initial"},
        )
wall_time = widgets.IntText(
            description="wall time",
            value = 1800,
            style = {"description_width": "initial"},
        )

block = widgets.VBox(children = [resources,
                                 account,
                                 wall_time
                                 ])
block


In [ ]:
pre = presettings.Presettings(INVERSION_WORKLFOW,
                              parameters = {'params':inv,
                                            'locations':sq})
pre

In [ ]:
def prepare_inversion():

    prepend_text_ = f'#SBATCH --account={account.value}'

    curent_names = make_query.get_extra_(INVERSION_WORKLFOW, name=None)
    if not inv.inv_name.value or inv.inv_name.value in curent_names:
        print('Invalid or empty run.str')
        return
    
    inverstion_parameters = inv.construct_dict()
    filter_dict = get_dict_from_filter()

    extras_dict = {k:{x.description:x.value for x in v} for k,v in sq.site_extras.items()}
    for k in extras_dict.keys():
        extras_dict[k]['bg.obs.fn'] = sq.available_obs_list[extras_dict[k]['bg.obs.fn']].attributes["remote_path"]

    for i in sq.list_info_obs:
        i.update(extras_dict[i['name']])
        i.update(filter_dict[i['name']])

    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})

    #other params
    inverstion_parameters.update({'para': sq.species.value,
                                'domain':sq.domain.value,
                                'transport.model':sq.model.value,
                                'res.dir':'./'})
    
    #test conversion
    inverstion_parameters = {
    key.replace(".", "_"): value for key, value in inverstion_parameters.items()
}
    remotes_dictionary = {k[:-3].replace('-','_'):v 
                         for k,v in sq.list_remotes.items()}
    
    #selected obs format change
    reformated_observations = {k.replace('-','_'):v 
                               for k,v in sq.selected_obs.items()}
    
    #builder starts
    builder = INVERSION_WORKLFOW.get_builder()
    builder.inversion_code = orm.load_code(resources.codes.value)
    builder.remotes = remotes_dictionary
    builder.observations =  reformated_observations
    builder.inversioncalc.metadata.options.custom_scheduler_commands = prepend_text_
    builder.inversioncalc.metadata.options.max_wallclock_seconds = wall_time.value
    builder.inv_params = orm.Dict(inverstion_parameters)
    builder.date_range = orm.Str(sq.date_range.value)
    builder.chunk = orm.Str(inv.chunk.value)
    builder.chunk_w = orm.Str(inv.chunk_w.value)
    builder.inversioncalc.metadata.options.stash = {
        'source_list': utils.read_yaml_data(path_config,
                                            names ='INVERSION')['INVERSION'],
        'target_base': resources.stash_address.value,
        'stash_mode': common.StashMode.COPY.value,
    }
    
    if pre.save_settings_b.value:
        builder.name = inv.inv_name.value

    return builder


In [ ]:
btn_submit_ = awb.SubmitButtonWidget(
    INVERSION_WORKLFOW,
    inputs_generator = prepare_inversion,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_.btn_submit.button_style="success"
display(btn_submit_, out)